In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

pd.options.display.max_columns = None
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve, auc, precision_recall_curve, average_precision_score, precision_score, recall_score, f1_score

from xgboost import XGBClassifier
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials

In [51]:
X,y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [52]:
space = {
    'max_depth': hp.choice('max_depth', range(1, 13)),
    'learning_rate': hp.quniform('learning_rate', 0.025, 0.5, 0.025),
    'n_estimators': hp.choice('n_estimators', range(100, 1000,100)),
    'gamma': hp.quniform('gamma', 0.5, 1, 0.05),
    'min_child_weight': hp.quniform('min_child_weight', 1, 6, 1),
    'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
    'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
    'colsample_bylevel': hp.quniform('colsample_bylevel', 0.5, 1, 0.05),
    'colsample_bynode': hp.quniform('colsample_bynode', 0.5, 1, 0.05),
    'reg_alpha': hp.quniform('reg_alpha', 0, 1, 0.05),
    'reg_lambda': hp.quniform('reg_lambda', 0, 1, 0.05),
}
def objective(space):
    evaluation = [( X_train, y_train), ( X_test, y_test)]
    model = XGBClassifier(
        max_depth = space['max_depth'],
        learning_rate = space['learning_rate'],
        n_estimators = space['n_estimators'],
        gamma = space['gamma'],
        min_child_weight = space['min_child_weight'],
        subsample = space['subsample'],
        colsample_bytree = space['colsample_bytree'],
        colsample_bylevel = space['colsample_bylevel'],
        colsample_bynode = space['colsample_bynode'],
        reg_alpha = space['reg_alpha'],
        reg_lambda = space['reg_lambda'],
        n_jobs = -1,
        random_state = 42,
        eval_metric = 'auc',
        early_stopping_rounds=10,
    )
    model.fit(
        X_train, y_train,
        eval_set=evaluation, 
        verbose=10
    )
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy = {}".format(accuracy))
    return {'loss': -accuracy, 'status': STATUS_OK}

In [54]:
trials = Trials()
best = fmin(
  fn=objective,
  space=space,
  algo=tpe.suggest,
  max_evals=100,
  trials=trials,
  rstate=np.random.default_rng(42)
)

[0]	validation_0-auc:0.89350	validation_1-auc:0.85458  
[10]	validation_0-auc:0.98187	validation_1-auc:0.94621 
[20]	validation_0-auc:0.98931	validation_1-auc:0.95524 
[30]	validation_0-auc:0.99384	validation_1-auc:0.96096 
[40]	validation_0-auc:0.99561	validation_1-auc:0.96367 
[50]	validation_0-auc:0.99672	validation_1-auc:0.96869 
[60]	validation_0-auc:0.99727	validation_1-auc:0.97340 
[67]	validation_0-auc:0.99767	validation_1-auc:0.97210 
Accuracy = 0.9                                         
[0]	validation_0-auc:0.69097	validation_1-auc:0.69289              
[10]	validation_0-auc:0.79450	validation_1-auc:0.78202             
[20]	validation_0-auc:0.83791	validation_1-auc:0.82507             
[30]	validation_0-auc:0.85818	validation_1-auc:0.84178             
[37]	validation_0-auc:0.86139	validation_1-auc:0.84760             
Accuracy = 0.755                                                   
[0]	validation_0-auc:0.72061	validation_1-auc:0.74594              
[10]	validation_0-au

In [ ]:
best

{'colsample_bylevel': 0.65,
 'colsample_bynode': 0.8500000000000001,
 'colsample_bytree': 0.7000000000000001,
 'gamma': 0.8,
 'learning_rate': 0.375,
 'max_depth': 9,
 'min_child_weight': 4.0,
 'n_estimators': 23,
 'reg_alpha': 0.45,
 'reg_lambda': 0.6000000000000001,
 'subsample': 0.9500000000000001}

In [ ]:
# train the model with the best parameters
model = XGBClassifier(
    max_depth = best['max_depth'],
    learning_rate = best['learning_rate'],
    n_estimators = best['n_estimators'],
    gamma = best['gamma'],
    min_child_weight = best['min_child_weight'],
    subsample = best['subsample'],
    colsample_bytree = best['colsample_bytree'],
    colsample_bylevel = best['colsample_bylevel'],
    colsample_bynode = best['colsample_bynode'],
    reg_alpha = best['reg_alpha'],
    reg_lambda = best['reg_lambda'],
    n_jobs = -1,
    random_state = 42,
)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy = {}".format(accuracy_score(y_test, y_pred)))


Accuracy = 0.9


In [ ]:
# get top 10 trials
top_trials = sorted(trials.results, key = lambda x: x['loss'], reverse=True)[:10]
top_trials

[{'loss': -0.785, 'status': 'ok'},
 {'loss': -0.81, 'status': 'ok'},
 {'loss': -0.81, 'status': 'ok'},
 {'loss': -0.825, 'status': 'ok'},
 {'loss': -0.835, 'status': 'ok'},
 {'loss': -0.84, 'status': 'ok'},
 {'loss': -0.845, 'status': 'ok'},
 {'loss': -0.855, 'status': 'ok'},
 {'loss': -0.87, 'status': 'ok'},
 {'loss': -0.87, 'status': 'ok'}]

In [ ]:
# top 10 params and accuracy
dict(trials.trials[0]['misc']['vals'])

{'colsample_bylevel': [0.5],
 'colsample_bynode': [0.7000000000000001],
 'colsample_bytree': [1.0],
 'gamma': [0.7000000000000001],
 'learning_rate': [0.42500000000000004],
 'max_depth': [5],
 'min_child_weight': [2.0],
 'n_estimators': [892],
 'reg_alpha': [0.6000000000000001],
 'reg_lambda': [0.15000000000000002],
 'subsample': [0.8500000000000001]}

In [57]:
results = list(map(lambda trial: dict({
    'opt_val': -trial['result']['loss'],
    'params': dict(trial['misc']['vals'])
}), trials.trials))

In [59]:
results.sort(key=lambda x: x['opt_val'], reverse=True)

In [64]:
print('Top 10 trials:')
for i in range(10):
    print('Trial {}:'.format(i+1))
    print('Optimal Value: {}'.format(results[i]['opt_val']))
    print('Parameters: {}'.format(results[i]['params']))
    print('')

Top 10 trials:
Trial 1:
Optimal Value: 0.95
Parameters: {'colsample_bylevel': [1.0], 'colsample_bynode': [0.7000000000000001], 'colsample_bytree': [0.5], 'gamma': [0.7000000000000001], 'learning_rate': [0.30000000000000004], 'max_depth': [2], 'min_child_weight': [6.0], 'n_estimators': [8], 'reg_alpha': [0.75], 'reg_lambda': [0.25], 'subsample': [0.9]}

Trial 2:
Optimal Value: 0.935
Parameters: {'colsample_bylevel': [0.8], 'colsample_bynode': [0.8], 'colsample_bytree': [0.6000000000000001], 'gamma': [0.75], 'learning_rate': [0.325], 'max_depth': [2], 'min_child_weight': [5.0], 'n_estimators': [2], 'reg_alpha': [0.6000000000000001], 'reg_lambda': [0.75], 'subsample': [0.65]}

Trial 3:
Optimal Value: 0.935
Parameters: {'colsample_bylevel': [0.9500000000000001], 'colsample_bynode': [0.65], 'colsample_bytree': [0.6000000000000001], 'gamma': [0.65], 'learning_rate': [0.35000000000000003], 'max_depth': [2], 'min_child_weight': [6.0], 'n_estimators': [8], 'reg_alpha': [0.45], 'reg_lambda': [0.